## Energy intensity calculation for the `Raw` data-sheet

The Energy Intensity calculations/formulas weren't included in the xlxs file, so this is an attempt to replicate those numbers (at least the values in the `Raw` tables).

First, the total energy consumed is calculated according to the formula:

$E_{\mathrm{total}} = \mathrm{Electricity} + \mathrm{Natural\,Gas}\times\beta_1 + \mathrm{Fuel}\times\beta_2 + ... $

where $\beta_i$ are coefficients that convert the energy source into units of `equivalant kilo-watt hours` (ekWh). See Table 1 in https://www.nrcan.gc.ca/energy/publications/efficiency/buildings/5985

Next, the total energy consumed is divided by either the `Total Floor Area` (in the case of buildings) or by the `Annual Flow`, for water treatment facilities.

$\mathrm{Energy\,Intensity} = E_{\mathrm{total}}/\mathrm{Total\,Floor\,Area}$

or

$\mathrm{Energy\,Intensity} = E_{\mathrm{total}}/\mathrm{Annual\,Flow}$

In [1]:
import pandas as pd
import numpy as np

Function to perform the Energy Intensity calculation, also taking care of the unit conversions:

In [236]:
def calculate_energy_intensity(row):
    def zero_nulls(x):
        if pd.isnull(x):
            return 0
        else:
            return x
    
    #calculate the equivalant kilo watt hours
    ekWh  = (1.0)     * zero_nulls(row['Electricity'])
    ekWh += (10.6273) * zero_nulls(row['Natural Gas'])
    ekWh += (10.74)   * zero_nulls(row['Fuel Oil 1 & 2'])
    ekWh += (11.25)   * zero_nulls(row['Fuel Oil 4 & 6'])
    ekWh += (7.09)    * zero_nulls(row['Propane'])
    ekWh += (3876.7)  * zero_nulls(row['Wood'])

    #Facilities related to the treatment of water
    if row['Annual Flow'] > 0:
        ekWh_per_ML = ekWh/row['Annual Flow']
        GJ_per_ML   = ekWh_per_ML * 0.0036  #0.0036 GJ/ekWh
        return GJ_per_ML, ekWh_per_ML

    #Calculation per square footage
    
    sqft = 0
    if row['Unit1'] in ['Square feet','Square Feet','square feet',u'pieds carrés',u'Pieds carrés','Sq Ft']:
        sqft = row['Total Floor Area']
    elif row['Unit1'] in ['Square meters','square meters','Square Meters']:
        sqft = row['Total Floor Area'] * 10.7639
    
    if sqft > 0:
        ekWh_per_sqft = ekWh/sqft
        GJ_per_m2 = ekWh_per_sqft * 0.0036/0.092903 #0.0036 GJ/ekWh, and 0.092903 m2/sqft
        return ekWh_per_sqft, GJ_per_m2
    else:
        return None, None

In [71]:
excel_file = pd.ExcelFile('bps_energy_consumption_2011_2012.xlsx', encoding='utf-8')
raw_2011 = excel_file.parse('2011 Raw')

##Examples:

A record with `Electricity`, `Natural Gas`, and `Total Floor Area` values:

In [141]:
raw_2011.ix[2367]

Organization Name                                                 Alexandra Hospital
Sector Name                                                          Public Hospital
SubSector Name                                                         Acute/Chronic
Operation Name                                                    Alexandra Hospital
Operation Type                         Administrative offices and related facilities
Address                                                              29 Noxon Street
City                                                                       Ingersoll
ZIP/Postal Code                                                              N5C 3V6
Total Floor Area                                                              3813.3
Unit1                                                                  Square meters
Annual Flow                                                                      NaN
Average Hours Per Week                                           

Results of the calculation: 74.6 ekWh/sqft, and 2.89 GJ/m2:

In [235]:
calculate_energy_intensity(raw_2011.ix[2367])

(74.617655462978234, 2.8914411769988226)

Now an example with `Annual Flow`:

In [242]:
raw_2011.ix[19]

Organization Name                                    Regional Municipality of Durham
Sector Name                                                                Municipal
SubSector Name                                                          Municipality
Operation Name                                                     Duffin Creek WPCP
Operation Type                         Facilities related to the treatment of sewage
Address                                                                901 McKay Rd.
City                                                                       Pickering
ZIP/Postal Code                                                              L1W 3A3
Total Floor Area                                                                 NaN
Unit1                                                                            NaN
Annual Flow                                                                 123041.8
Average Hours Per Week                                           

The calculation gives 3.92 GJ/Mega Litres and 1090.2 ekWh/Mega Litres:

In [243]:
calculate_energy_intensity(raw_2011.ix[19])

(3.9248987284349264, 1090.2496467874796)